In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 99.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 115.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForTokenClassification,AutoTokenizer,pipeline
model = AutoModelForTokenClassification.from_pretrained('uer/roberta-base-finetuned-cluener2020-chinese')
tokenizer = AutoTokenizer.from_pretrained('uer/roberta-base-finetuned-cluener2020-chinese', model_max_length=512)
ner = pipeline('ner', model=model, tokenizer=tokenizer)

In [4]:
def ner_by_cluener(title):
  ner_result = ner(title)
  single_name_str = ""
  name_list = []
  for char_dic in ner_result:
    if char_dic['entity'] == "B-name" or "I-name":
      if (single_name_str=="" and char_dic['entity'] == "B-name") or char_dic['entity'] == "I-name":
        single_name_str+=char_dic['word']
      elif char_dic['entity'] == "B-name":
        name_list.append(single_name_str)
        single_name_str = char_dic['word']
  if single_name_str != "":
    name_list.append(single_name_str)
  name_list = [i for i in name_list if len(i)>1]
  name_list = list(set(name_list))
  return ";".join(name_list)

In [9]:
import csv

output = []
count = 0
print("Start tagging...")
with open("/content/drive/MyDrive/tag_person_names_by_cluener/input.txt", 'r', encoding='utf-8') as f_reader:
  input_data = [i.rstrip('\n') for i in f_reader.readlines()]
  count_max = len(input_data)
  for title in input_data:
    if count%2000 == 0 and count != 0:
        print("{:.2f}% finished...".format(count/count_max*100))
        with open(f"/content/drive/MyDrive/tag_person_names_by_cluener/temp_output/output_{count}.csv", "w", encoding='utf-8', newline='') as f_writer:
            csv_writer = csv.writer(f_writer)
            csv_writer.writerows(output)
            output = []
    personname_str = ner_by_cluener(title)
    output.append([title, personname_str])
    count+=1
if output != []:
  with open(f"/content/drive/MyDrive/tag_person_names_by_cluener/temp_output/output_{count}.csv", "w", encoding='utf-8', newline='') as f_writer:
    csv_writer = csv.writer(f_writer)
    csv_writer.writerows(output)

Start tagging...
10.53% finished...
21.06% finished...
31.58% finished...
42.11% finished...
52.64% finished...
63.17% finished...
73.70% finished...
84.22% finished...
94.75% finished...


In [10]:
import csv
from glob import glob
print("Combining output...")
count = 0
output = []
with open("/content/drive/MyDrive/tag_person_names_by_cluener/output.csv", "w", encoding='utf-8', newline='') as f_writer:
    csv_writer = csv.writer(f_writer)
    for filename in glob("/content/drive/MyDrive/tag_person_names_by_cluener/temp_output/*.csv"):
        with open(filename, newline='') as f:
            reader = csv.reader(f)
            for row in reader:
                csv_writer.writerow(row)

print("Removing temp data...")
!rm -r /content/drive/MyDrive/tag_person_names_by_cluener/temp_output/*.csv

print("Done!")

Combining output...
Removing temp data...
Done!
